# Neural Network Smartphone Activity Detector

In this activity, you will train a neural network to use smartphone data to predict the activity of the user. 

This dataset has already been separated into input features and target activities. Additional information on the dataset can be found here. 

http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions

### Data Pre-Processing

Prepare the data for the neural network. This includes splitting the data into a training and testing dataset, Scaling the data, and encoding the categorical target values

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Upload data to Colab
from google.colab import files

csv_file = files.upload()

Saving features.csv to features (1).csv


In [3]:
# Read the input features into `X`
X = pd.read_csv("features.csv")
X.head()

,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,tBodyAcc-Max-2,tBodyAcc-Max-3,tBodyAcc-Min-1,tBodyAcc-Min-2,tBodyAcc-Min-3,tBodyAcc-SMA-1,tBodyAcc-Energy-1,tBodyAcc-Energy-2,tBodyAcc-Energy-3,tBodyAcc-IQR-1,tBodyAcc-IQR-2,tBodyAcc-IQR-3,tBodyAcc-ropy-1,tBodyAcc-ropy-1.1,tBodyAcc-ropy-1.2,tBodyAcc-ARCoeff-1,tBodyAcc-ARCoeff-2,tBodyAcc-ARCoeff-3,tBodyAcc-ARCoeff-4,tBodyAcc-ARCoeff-5,tBodyAcc-ARCoeff-6,tBodyAcc-ARCoeff-7,tBodyAcc-ARCoeff-8,tBodyAcc-ARCoeff-9,tBodyAcc-ARCoeff-10,tBodyAcc-ARCoeff-11,tBodyAcc-ARCoeff-12,tBodyAcc-Correlation-1,tBodyAcc-Correlation-2,tBodyAcc-Correlation-3,...,fBodyAccJerkMag-Energy-1,fBodyAccJerkMag-IQR-1,fBodyAccJerkMag-ropy-1,fBodyAccJerkMag-MaxInds-1,fBodyAccJerkMag-MeanFreq-1,fBodyAccJerkMag-Skewness-1,fBodyAccJerkMag-Kurtosis-1,fBodyGyroMag-Mean-1,fBodyGyroMag-STD-1,fBodyGyroMag-Mad-1,fBodyGyroMag-Max-1,fBodyGyroMag-Min-1,fBodyGyroMag-SMA-1,fBodyGyroMag-Energy-1,fBodyGyroMag-IQR-1,fBodyGyroMag-ropy-1,fBodyGyroMag-MaxInds-1,fBodyGyroMag-MeanFreq-1,fBodyGyroMag-Skewness-1,fBodyGyroMag-Kurtosis-1,fBodyGyroJerkMag-Mean-1,fBodyGyroJerkMag-STD-1,fBodyGyroJerkMag-Mad-1,fBodyGyroJerkMag-Max-1,fBodyGyroJerkMag-Min-1,fBodyGyroJerkMag-SMA-1,fBodyGyroJerkMag-Energy-1,fBodyGyroJerkMag-IQR-1,fBodyGyroJerkMag-ropy-1,fBodyGyroJerkMag-MaxInds-1,fBodyGyroJerkMag-MeanFreq-1,fBodyGyroJerkMag-Skewness-1,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1
0,0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,-0.744893,-0.648447,0.841796,0.708440,0.651716,-0.975752,-0.999950,-0.999888,-0.998014,-0.993999,-0.991980,-0.970970,-0.547095,-0.700974,-0.622697,0.921884,-0.719483,0.342168,-0.161318,0.266049,-0.274351,0.267205,-0.020958,0.382610,-0.501748,0.512463,-0.206337,0.376778,0.435172,0.660199,...,-0.999918,-0.991736,-1.0,-0.936508,0.349260,-0.517127,-0.801006,-0.980135,-0.961301,-0.974129,-0.956013,-0.989894,-0.980135,-0.999240,-0.992673,-0.701291,-1.000000,-0.132480,0.565697,0.363478,-0.991994,-0.990877,-0.990169,-0.992521,-0.991044,-0.991994,-0.999937,-0.990537,-0.871306,-1.000000,-0.012236,-0.314848,-0.713308,-0.112754,0.030400,-0.464761,-0.018446,-0.841559,0.179913,-0.051718
1,0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,-0.736338,-0.712415,0.838758,0.708440,0.659340,-0.987427,-0.999993,-0.999826,-0.999411,-0.998918,-0.985482,-0.973481,-0.781973,-0.534604,-0.593165,0.607435,-0.266783,0.275882,0.200417,0.131266,-0.149017,0.292436,-0.192986,0.217496,-0.089175,0.059909,-0.236609,-0.012696,-0.072711,0.578649,...,-0.999867,-0.991506,-1.0,-0.841270,0.533688,-0.625993,-0.898311,-0.988296,-0.983313,-0.982951,-0.987406,-0.992134,-0.988296,-0.999811,-0.993996,-0.720683,-0.948718,-0.268979,-0.364219,-0.723724,-0.995857,-0.996580,-0.995671,-0.996939,-0.994436,-0.995857,-0.999981,-0.994623,-1.000000,-1.000000,0.202804,-0.603199,-0.860677,0.053477,-0.007435,-0.732626,0.703511,-0.845092,0.180261,-0.047436
2,0.039978,-0.005153,-0.022651,-0.995482,-0.977314,-0.984760,-0.996415,-0.975835,-0.985973,-0.798477,-0.736338,-0.712415,0.834002,0.705008,0.674551,-0.988528,-0.999972,-0.999719,-0.999803,-0.996898,-0.976781,-0.986754,-0.688176,-0.520514,-0.593165,0.272262,-0.056424,0.322283,-0.273292,0.037180,-0.133612,0.332487,-0.240491,0.348733,-0.195409,0.229436,-0.316816,-0.123889,-0.181137,0.608219,...,-0.999845,-0.987029,-1.0,-0.904762,0.661975,-0.725887,-0.926663,-0.989255,-0.986019,-0.984533,-0.991701,-0.995857,-0.989255,-0.999854,-0.993256,-0.736521,-0.794872,-0.212429,-0.564868,-0.874594,-0.995034,-0.995308,-0.994868,-0.996133,-0.995863,-0.995034,-0.999973,-0.993834,-1.000000,-0.555556,0.440079,-0.404427,-0.761847,-0.118559,0.177899,0.100699,0.808529,-0.849

In [4]:
# Upload data to Colab
csv_file = files.upload()

Saving target.csv to target.csv


In [5]:
# Read the target values into `y`
y = pd.read_csv("target.csv")
y.head()

,activity
0,standing
1,standing
2,standing
3,standing
4,standing


In [6]:
y.activity.value_counts()

standing              1423
laying                1413
sitting               1293
walking               1226
walking_upstairs      1073
walking_downstairs     987
stand_to_lie            90
sit_to_lie              75
lie_to_sit              60
lie_to_stand            57
stand_to_sit            47
sit_to_stand            23
Name: activity, dtype: int64

In [7]:
# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
# Scale the training and testing input features using StandardScaler
X_scaler = StandardScaler()
X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Apply One-hot encoding to the target labels
enc = OneHotEncoder()
enc.fit(y_train)

encoded_y_train = enc.transform(y_train).toarray()
encoded_y_test = enc.transform(y_test).toarray()
encoded_y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

# Build a Deep Neural Network

In [10]:
# Create a sequential model
model = Sequential()

In [11]:
# Add the first layer where the input dimensions are the 561 columns of the training data
model.add(Dense(100, activation='relu', input_dim=X_train_scaled.shape[1]))

In [12]:
# The output layer has 12 columns that are one-hot encoded
y_train.activity.value_counts()
number_outputs = 12

In [13]:
# Add output layer using 12 output nodes
model.add(Dense(number_outputs, activation="softmax"))

In [14]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [15]:
# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               56200     
                                                                 
 dense_1 (Dense)             (None, 12)                1212      
                                                                 
Total params: 57,412
Trainable params: 57,412
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Use the training data to fit (train) the model
# @NOTE: Experiment with the number of training epochs to find the minimum iterations required to achieve a good accuracy
model.fit(
    X_train_scaled,
    encoded_y_train,
    epochs=30,
    shuffle=True,
    verbose=2
)

Epoch 1/30
183/183 - 1s - loss: 0.3757 - accuracy: 0.8748 - 1s/epoch - 6ms/step
Epoch 2/30
183/183 - 0s - loss: 0.1250 - accuracy: 0.9579 - 348ms/epoch - 2ms/step
Epoch 3/30
183/183 - 0s - loss: 0.0864 - accuracy: 0.9691 - 313ms/epoch - 2ms/step
Epoch 4/30
183/183 - 0s - loss: 0.0657 - accuracy: 0.9782 - 332ms/epoch - 2ms/step
Epoch 5/30
183/183 - 0s - loss: 0.0556 - accuracy: 0.9808 - 305ms/epoch - 2ms/step
Epoch 6/30
183/183 - 0s - loss: 0.0505 - accuracy: 0.9827 - 330ms/epoch - 2ms/step
Epoch 7/30
183/183 - 0s - loss: 0.0594 - accuracy: 0.9789 - 312ms/epoch - 2ms/step
Epoch 8/30
183/183 - 0s - loss: 0.0361 - accuracy: 0.9871 - 328ms/epoch - 2ms/step
Epoch 9/30
183/183 - 0s - loss: 0.0402 - accuracy: 0.9845 - 313ms/epoch - 2ms/step
Epoch 10/30
183/183 - 0s - loss: 0.0295 - accuracy: 0.9900 - 325ms/epoch - 2ms/step
Epoch 11/30
183/183 - 0s - loss: 0.0258 - accuracy: 0.9918 - 320ms/epoch - 2ms/step
Epoch 12/30
183/183 - 0s - loss: 0.0187 - accuracy: 0.9940 - 336ms/epoch - 2ms/step
Epoc

# Evaluate the Model

In [17]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, encoded_y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 - 0s - loss: 0.0711 - accuracy: 0.9743 - 233ms/epoch - 4ms/step
Normal Neural Network - Loss: 0.07110808044672012, Accuracy: 0.9742533564567566


In [18]:
# Make predictions
predicted = model.predict(X_test_scaled)
predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test.activity.values,
    "Predicted": predicted
})
results.head(10)

,Actual,Predicted
0,walking_upstairs,walking_upstairs
1,laying,laying
2,sitting,sitting
3,sitting,sitting
4,walking,walking
5,sitting,sitting
6,lie_to_sit,lie_to_sit
7,walking_downstairs,walking_downstairs
8,laying,laying
9,sitting,sitting


In [19]:
# Print the Classification Report
from sklearn.metrics import classification_report
print(classification_report(results.Actual, results.Predicted))

                    precision    recall  f1-score   support

            laying       1.00      1.00      1.00       355
        lie_to_sit       1.00      0.80      0.89        15
      lie_to_stand       0.79      1.00      0.88        11
        sit_to_lie       0.85      0.74      0.79        23
      sit_to_stand       1.00      1.00      1.00         4
           sitting       0.96      0.95      0.95       337
      stand_to_lie       0.72      0.72      0.72        18
      stand_to_sit       0.87      0.87      0.87        15
          standing       0.95      0.97      0.96       367
           walking       1.00      0.99      0.99       300
walking_downstairs       0.99      1.00      1.00       230
  walking_upstairs       0.99      1.00      0.99       267

          accuracy                           0.97      1942
         macro avg       0.93      0.92      0.92      1942
      weighted avg       0.97      0.97      0.97      1942

